In [7]:
import numpy as np
from sktime.datasets import load_from_tsfile
from scipy.ndimage import gaussian_filter1d

# ============================================================
# 1. Parser UEA universal
# ============================================================

def load_uea_ts(path):
    X = []
    y = []
    in_data = False

    with open(path, "r") as f:
        for line in f:
            line = line.strip()

            if not line:
                continue

            if line.startswith("@"):
                if line.lower().startswith("@data"):
                    in_data = True
                continue

            if not in_data:
                continue

            parts = line.split(":")
            dims = parts[:-1]
            label = parts[-1]

            channels = []
            for d in dims:
                vals = d.split(",")

                if vals[0].startswith("dim_"):
                    vals = vals[1:]

                arr = np.array(list(map(float, vals)), dtype=np.float32)
                channels.append(arr)

            X.append(np.stack(channels, axis=0))
            y.append(label)

    return np.stack(X, axis=0), y


# =====================================================================
# 2. Cargar BasicMotions (TRAIN + TEST)
# =====================================================================

def load_full_basicmotions(train_path, test_path):
    X_train, y_train = load_uea_ts(train_path)
    X_test, y_test = load_uea_ts(test_path)

    X_all = np.concatenate([X_train, X_test], axis=0)
    y_all = y_train + y_test

    return X_all, y_all


# =====================================================================
# 3. Estadísticas por canal
# =====================================================================

def compute_channel_stats(X_np):
    stats = []
    C = X_np.shape[1]

    for c in range(C):
        ch = X_np[:, c, :].flatten()
        stats.append({
            "min": float(np.min(ch)),
            "max": float(np.max(ch)),
            "mean": float(np.mean(ch)),
            "std": float(np.std(ch)),
        })

    return stats


# =====================================================================
# 4. Generación sintética
# =====================================================================

def generate_synthetic_series(stats, T=100, smooth_sigma=2.0):
    C = len(stats)
    syn = np.zeros((C, T), dtype=np.float32)

    for c in range(C):
        s = stats[c]

        # base random walk
        x = np.zeros(T, dtype=np.float32)
        x[0] = np.random.uniform(s["mean"] - s["std"], s["mean"] + s["std"])

        for t in range(1, T):
            step = np.random.normal(0, s["std"] * 0.08)
            x[t] = x[t - 1] + step

        # suavizar
        x = gaussian_filter1d(x, sigma=smooth_sigma)
        x = np.clip(x, s["min"], s["max"])

        syn[c] = x

    return syn


# =====================================================================
# 5. Escritura TS file
# =====================================================================

def write_ts(X_list, y_list, out_path):

    with open(out_path, "w") as f:

        labels = sorted(set(y_list))

        f.write("@problemName SyntheticBasicMotions\n")
        f.write("@timeStamps false\n")
        f.write("@univariate false\n")
        f.write("@dimensions 6\n")
        f.write("@equalLength true\n")
        f.write("@seriesLength 100\n")
        f.write("@classLabel true " + " ".join(labels) + "\n")
        f.write("@data\n")

        for X, y in zip(X_list, y_list):
            dims_str = [",".join(str(v) for v in X[c]) for c in range(6)]
            row = ":".join(dims_str) + f":{y}\n"
            f.write(row)

    print(f"✔ Synthetic TS file generado: {out_path}")


# =====================================================================
# 6. MAIN
# =====================================================================
dataset_folder = "../Dataset/UEA/Multivariate_ts/BasicMotions/"
def generate_synthetic_basicmotions(
        n_samples=10,
        train_path=dataset_folder + "samples/BasicMotions_TRAIN.ts",
        test_path=dataset_folder + "samples/BasicMotions_TEST.ts"):
    X_real, y_real = load_full_basicmotions(train_path, test_path)
    stats = compute_channel_stats(X_real)

    X_syn = []
    y_syn = []
    labels = sorted(set(y_real))

    for _ in range(n_samples):
        X_syn.append(generate_synthetic_series(stats))
        y_syn.append(np.random.choice(labels))

    write_ts(X_syn, y_syn, dataset_folder + "SyntheticBasicMotions_TEST.ts")

In [8]:
generate_synthetic_basicmotions()


✔ Synthetic TS file generado: ../Dataset/UEA/Multivariate_ts/BasicMotions/SyntheticBasicMotions_TEST.ts
